In [1]:
import __init__

from collections import defaultdict
from functools import partial
from typing import Callable

import numpy as np
from cv2.typing import MatLike
from eval import cv2_to_pil, load_data, pil_to_cv2
from eval_with_ground import evaluate_image_pair_from_pil
from eval_without_ground import evaluate_image
from tqdm import tqdm

from common import DataSets
from dehaze import dehaze_and_enhance
from gan import gan_method

/home/rmc/csds490_project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
paired_samples = load_data()

100%|██████████| 7200/7200 [01:13<00:00, 97.77it/s] 


In [3]:
def eval(dataset: DataSets, method: Callable[[MatLike], MatLike]):
    samples = paired_samples[dataset]
    results = defaultdict(list)
    for sample in tqdm(samples):
        low_image = sample["low_image"]
        high_image = sample.get("high_image")
        # print(low_image, high_image)
        if low_image is not None:
            reference_img = cv2_to_pil(method(pil_to_cv2(low_image)))
            # reference_img.show()
            if high_image is not None:
                scores = evaluate_image_pair_from_pil(reference_img, high_image)
                # Compute metrics here
            else:
                scores = evaluate_image(reference_img, low_image)
            for key, s in scores.items():
                results[key].append(s)
    return results

In [4]:
paired_samples.keys()

dict_keys(['lol_dataset', 'Dark_Face', 'LOL-v2'])

# dehaze

In [5]:
result = eval('lol_dataset', dehaze_and_enhance)

100%|██████████| 500/500 [00:19<00:00, 26.07it/s]


In [6]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 3876.0293 ± 3862.0581
PSNR: 14.2037 ± 4.2630
SSIM: 0.5057 ± 0.1516


In [7]:
result = eval('Dark_Face', dehaze_and_enhance)

100%|██████████| 6000/6000 [05:43<00:00, 17.49it/s]


In [8]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 0.9235 ± 6.6518
CEI: 1.8999 ± 0.4300


In [9]:
result = eval('LOL-v2', dehaze_and_enhance)

100%|██████████| 100/100 [00:03<00:00, 30.36it/s]


In [10]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 2107.1069 ± 1780.1074
PSNR: 16.2354 ± 3.3389
SSIM: 0.5460 ± 0.1234


# Generative Adversarial Networks

In [4]:
dataset = 'lol_dataset'
result = eval(dataset, partial(gan_method, dataset))

100%|██████████| 500/500 [00:16<00:00, 29.71it/s]


In [5]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 5425.7422 ± 2223.0605
PSNR: 11.1039 ± 1.6264
SSIM: 0.5946 ± 0.1382


In [5]:
dataset = "LOL-v2"
result = eval(dataset, partial(gan_method, dataset))

100%|██████████| 100/100 [00:04<00:00, 23.32it/s]


In [6]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 5344.8062 ± 2327.2822
PSNR: 11.2398 ± 1.8266
SSIM: 0.0966 ± 0.0227


In [ ]:
dataset = "Dark_Face"
result = eval(dataset, partial(gan_method, dataset))

In [ ]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")